Hi everyone! This is the notebook we will use tf.Keras to train/evaluate/predict a classifer for CIFAR-10 with convolutional layers. With tf.Keras things become very simple all we have to do is to combine codes from previous notebooks!

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import utils
print(tf.__version__)

In [ ]:
class_names_cifar10 = ['Airplane','Automobile','Bird','Cat','Deer',
                       'Dog','Frog','Horse','Ship','Truck']

Preparing Data

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
trainDataset = tf.data.Dataset.from_tensor_slices(x_train)

In [ ]:
utils.show_images(x_train[1:10],y_train[1:10],class_names_cifar10)

Preparing Model

In [ ]:
cnnModel = keras.Sequential()
cnnModel.add(layers.Conv2D(input_shape=[32,32,3],filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnnModel.add(layers.Dropout(0.25))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnnModel.add(layers.Dropout(0.25))
cnnModel.add(layers.Flatten())
cnnModel.add(layers.Dense(units = 512, activation = 'relu'))
cnnModel.add(layers.Dense(units = 10, activation = 'softmax'))
cnnModel.summary()

Compiling Model

In [ ]:
cnnModel.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Training Procedure

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
history = cnnModel.fit(x_train, y_train, batch_size=64, epochs=8, validation_split=0.1, callbacks = callbacks)

In [ ]:
# history record the accuracy/loss on training set/validation set after every epoch 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'valivation'], loc='upper left')
plt.show()

In [ ]:
result = cnnModel.evaluate(x_test, y_test)

In [ ]:
result

You can change the metrics by recompile the model without retraining. Compile defines the loss function, the optimizer and the metrics. That's all.It has nothing to do with the weights and you can compile a model as many times as you want without causing any problem to pretrained weights.

In [ ]:
cnnModel.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[['accuracy', 'mse']])

In [ ]:
result = cnnModel.evaluate(x_test, y_test)

In [ ]:
result

Prediction Phase

In [ ]:
predict_result = tf.argmax(cnnModel.predict(x_test[:9]),axis=1)

In [ ]:
predict_result.numpy()

In [ ]:
utils.show_images(x_test[0:9],y_test[0:9],class_names_cifar10, predict_result)